In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

In [15]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path+"cyl_val/", "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist
# cylinder max konz (per cell)
def getData_(cid,gId,path2file):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

exceptPset =[]
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    for pSet in allPSets:
        print(pSet,end =", ")

        #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        path2file = 'PDECSbnone_1440_'+str(pSet)+'_10to11_20mn_0s_5/'
        
        rr = getCylIdx(path2file)
        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        for cid in range(nKonz,volIdx):
            nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits
        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]
        
        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", dtype =int)
        id_roots = id_orgs[np.where(ot_orgs==2)]
        
        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])
        
        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))
        
        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])
            
            
        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens

    return GiniAll #cid pSet rr
    

In [16]:
GiniAll = getData()

0, ( 232 ), 1, ( 231 ), 2, ( 229 ), 3, ( 232 ), 4, ( 235 ), 5, ( 213 ), 6, ( 219 ), 7, ( 237 ), 8, ( 227 ), 9, ( 234 ), 10, ( 232 ), 11, ( 227 ), 12, ( 219 ), 13, ( 232 ), 14, ( 231 ), 15, ( 234 ), 16, ( 232 ), 17, ( 215 ), 18, ( 232 ), 19, ( 235 ), 20, ( 222 ), 21, ( 213 ), 22, ( 232 ), 23, ( 235 ), 24, ( 236 ), 25, ( 234 ), 26, ( 232 ), 27, ( 237 ), 28, ( 229 ), 29, ( 232 ), 30, ( 232 ), 31, ( 232 ), 32, ( 229 ), 33, ( 232 ), 34, ( 237 ), 35, ( 235 ), 36, ( 234 ), 37, ( 235 ), 38, ( 232 ), 39, ( 231 ), 40, ( 215 ), 41, ( 236 ), 42, ( 232 ), 43, ( 236 ), 44, ( 231 ), 45, ( 234 ), 46, ( 235 ), 47, ( 237 ), 48, ( 232 ), 49, ( 236 ), 50, ( 236 ), 51, ( 237 ), 52, ( 235 ), 53, ( 215 ), 54, ( 231 ), 55, ( 237 ), 56, ( 237 ), 57, ( 215 ), 58, ( 236 ), 59, ( 232 ), 60, ( 237 ), 61, ( 229 ), 62, ( 237 ), 63, ( 232 ), 64, ( 236 ), 65, ( 231 ), 66, ( 234 ), 67, ( 229 ), 68, ( 229 ), 69, ( 234 ), 70, ( 237 ), 71, ( 232 ), 72, ( 237 ), 73, ( 237 ), 74, ( 201 ), 75, ( 236 ), 76, ( 234 ), 77, ( 231

In [131]:
#assert np.array(GiniAll).shape == (nToGet, numPset, 232) # some have a couple of segments more

/tmp/ipykernel_1126/1273250247.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert np.array(GiniAll).shape == (nToGet, numPset, 232) #all same shape


AssertionError: 

In [17]:
np.array(GiniAll).shape

/tmp/ipykernel_682098/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 99)

In [18]:
# get param set
paramsets =  pd.read_csv('../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
paramsets['k_C,S'] /=paramsets['theta']
paramsets['k_O,S'] /=paramsets['theta']
paramsets.drop(labels='theta', inplace=True, axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','css2','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + paramsets.columns.tolist()+['pSet']
df = pd.DataFrame(columns=column_names)

for pset in allPSets:#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_to_duplicate_df = pd.DataFrame(paramsets.iloc[pset]).T
    duplicated_rows_df = rows_to_duplicate_df.loc[np.repeat(rows_to_duplicate_df.index.values, N)]
    # Duplicate the specific rows N times
    #print('init dupl row shape',duplicated_rows_df.shape)
    for jj, outVn in enumerate(outputvalsname):
        duplicated_rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    duplicated_rows_df['pSet'] = pset
    
    path2file = 'PDECSbnone_1440_'+str(pset)+'_10to11_20mn_0s_5/'
    simTime = get_last_line_as_numpy_array(path2file+"fpit_totalComputetime.txt", dtype =float)[0]
    
    duplicated_rows_df['simTime'] = simTime
    # Concatenate the original DataFrame with the duplicated specific rows
    df = pd.concat([df, duplicated_rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [30]:
df['simTime'] 

0        22865.345728
1        22865.345728
2        22865.345728
3        22865.345728
4        22865.345728
             ...     
20896    28032.934171
20897    28032.934171
20898    28032.934171
20899    28032.934171
20900    28032.934171
Name: simTime, Length: 20901, dtype: float64

In [19]:
df.to_pickle('./cyl10to11_pdecsb.pkl')

In [ ]:

path2file = 'noCss1none_1440_'+str(0)+'_10to11_20mn_0s_5/'
simTime = get_last_line_as_numpy_array(path2file+"totalComputetime.txt", dtype =float)[0]

In [ ]:
simTime

In [31]:
min(df['simTime'] )

9925.080730255693

In [20]:
df

,wat,cs,cl,coa,cod,cca,ccd,css2,co2,yX,...,Y_C,Y_O,CSS_max,k_sorp,CL_init,CS_init,pSet,simTime,co,cc
0,0.048045,0.000323,0.000926,0.000009,7.043014e-07,0.000010,5.704579e-15,6.779698e-05,0.000034,-0.298434,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,34448.755227,0.000010,0.000010
1,0.008128,0.001264,0.000916,0.000009,5.607989e-07,0.000011,1.376856e-18,1.032227e-04,0.000041,0.043922,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,34448.755227,0.000010,0.000011
2,0.010195,0.000862,0.000926,0.000009,5.778954e-07,0.000011,3.474045e-18,9.636330e-05,0.000040,-0.447197,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,34448.755227,0.000010,0.000011
3,0.012441,0.000954,0.000928,0.000009,5.778281e-07,0.000011,3.322379e-18,9.857022e-05,0.000040,-0.104340,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,34448.755227,0.000010,0.000011
4,0.011423,0.001000,0.000920,0.000009,5.777716e-07,0.000011,3.446562e-18,9.929696e-05,0.000040,-0.125120,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,34448.755227,0.000010,0.000011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22928,0.030768,0.000021,0.000671,0.000005,3.769675e-06,0.000003,6.320199e-06,3.534407e-07,0.000017,0.666116,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,21402.038798,0.000009,0.000010
22929,0.053793,0.000021,0.000671,0.000006,3.134948e-06,0.000004,5.751528e-06,4.287976e-07,0.000018,0.070774,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,21402.038798,0.000009,0.000010
22930,0.026067,0.000024,0.000672,0.000006,3.570893e-06,0.000004,5.813916e-06,4.350013e-07,0.000023,-0.583227,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,21402.038798,0.000009,0.000010
22931,0.111032,0.000017,0.000672,0.000003,6.119528e-06,0.000002,7.777945e-06,2.265992e-07,0.000006,-0.081112,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,21402.038798,0.000009,0.000009


In [21]:
df['css2']

0        6.779698e-05
1        1.032227e-04
2        9.636330e-05
3        9.857022e-05
4        9.929696e-05
             ...     
22928    3.534407e-07
22929    4.287976e-07
22930    4.350013e-07
22931    2.265992e-07
22932    4.738195e-07
Name: css2, Length: 22933, dtype: float64

In [22]:
print(min(df['css2']),max(df['css2']),np.mean(df['css2']))

1.5503365756135056e-09 0.00011841994628907898 1.6039859551602106e-05
